In [ ]:
# split iris data into train and test sets
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("dataset/iris.csv")

X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

train_df = X_train.copy()
train_df['class'] = y_train

test_df = X_test.copy()
test_df['class'] = y_test

train_df.to_csv("dataset/iris_train.csv", index=False)
test_df.to_csv("dataset/iris_test.csv", index=False)


In [ ]:
from watermarking_schemes.B2Mark import B2MarkWatermarkEmbedding
g_s = [2,3,4,5,6,7,8,9,10]

for g in g_s:
    for seed in range(10000,10100):
        wm = B2MarkWatermarkEmbedding(
            dataset="iris",
            seed=seed,
            g=g,
            gamma=1/2,
            secret_key="123",
            columns_of_interest=['sepal_length', 'sepal_width'],
            label_column='class'
        )
        wm.load_data('dataset/iris_train.csv')
        wm.process_data()
        wm.save_results(f'dataset/iris_train_{g}_{seed}.csv')



In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from collections import defaultdict


# ===== 1. 定义 MLP 模型 =====
class MLP(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=16, output_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x): 
        return self.net(x)

# ===== 2. FGSM 攻击函数 =====
def fgsm_attack(model, loss_fn, x, y, epsilon=0.1):
    x_adv = x.clone().detach().requires_grad_(True)
    output = model(x_adv)
    loss = loss_fn(output, y)
    model.zero_grad()
    loss.backward()
    # 沿梯度符号方向扰动
    x_adv = x_adv + epsilon * x_adv.grad.sign()
    # 适当裁剪，避免数值越界
    return torch.clamp(x_adv, -5, 5).detach()

# ===== 3. 训练函数 =====
def train_model(X_train, y_train, epochs=50, lr=0.01):
    model = MLP()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(epochs):
        model.train()
        out = model(X_train)
        loss = loss_fn(out, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

# ===== 4. 测试函数 =====
def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        preds = model(X).argmax(dim=1)
    return accuracy_score(y.numpy(), preds.numpy())

# ===== 5. 加载并预处理 CSV 数据 =====
def load_iris_data(csv_path):
    """ 返回预处理好的 (X_tensor, y_tensor) """
    df = pd.read_csv(csv_path)

    X = df.drop(columns=['class']).values
    y = df['class'].values

    # LabelEncoder 方便将字符串标签转为 int
    le = LabelEncoder()
    y = le.fit_transform(y)  # 0,1,2
    
    # 标准化
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    return X_tensor, y_tensor

def main():
    # 加载统一的测试集和训练集（不重新划分）
    X_test, y_test = load_iris_data("dataset/iris_test.csv")
    X_train, y_train = load_iris_data("dataset/iris_train.csv")

    # 保存每次的准确率
    acc_clean_list = []
    acc_fgsm_list = []

    for trial in range(100):
        model = train_model(X_train, y_train)
        
        acc_clean = evaluate_model(model, X_test, y_test)
        acc_clean_list.append(acc_clean)

        X_test_adv = fgsm_attack(model, nn.CrossEntropyLoss(), X_test, y_test, epsilon=0.1)
        acc_fgsm = evaluate_model(model, X_test_adv, y_test)
        acc_fgsm_list.append(acc_fgsm)

    # 统计结果
    acc_clean_mean = np.mean(acc_clean_list)
    acc_clean_std  = np.std(acc_clean_list)

    acc_fgsm_mean = np.mean(acc_fgsm_list)
    acc_fgsm_std  = np.std(acc_fgsm_list)

    # 打印
    print("=== Baseline (No Watermark, Trained 100x) ===")
    print(f"Clean Test Acc: {acc_clean_mean:.2%} ± {acc_clean_std:.2%}")
    print(f"FGSM  Test Acc: {acc_fgsm_mean:.2%} ± {acc_fgsm_std:.2%}")



    # 存储不同 g 下的 seed 结果
    results_by_g = defaultdict(list)

    g_s = [2, 3, 4, 5, 6, 7, 8, 9, 10]
    seeds = range(10000, 10100)

    for g in g_s:
        for seed in seeds:
            path = f"dataset/iris_train_{g}_{seed}.csv"
            try:
                X_train_wm, y_train_wm = load_iris_data(path)
            except FileNotFoundError:
                continue

            model = train_model(X_train_wm, y_train_wm)
            acc_clean = evaluate_model(model, X_test, y_test)

            # 使用水印模型生成自己对应的对抗样本
            X_test_adv = fgsm_attack(model, nn.CrossEntropyLoss(), X_test, y_test, epsilon=0.1)
            acc_adv = evaluate_model(model, X_test_adv, y_test)
            
            

            results_by_g[g].append((acc_clean, acc_adv))

    # 平均输出
    print("=== Watermarked Summary (grouped by g) ===")
    for g in sorted(results_by_g.keys()):
        arr = np.array(results_by_g[g])
        clean_mean = arr[:, 0].mean()
        adv_mean = arr[:, 1].mean()
        print(f"g={g:<2d} | Clean Acc: {clean_mean:.2%} | FGSM Acc: {adv_mean:.2%} | seeds: {len(arr)}")

if __name__ == "__main__":
    main()

=== Baseline (No Watermark, Trained 100x) ===
Clean Test Acc: 90.13% ± 4.11%
FGSM  Test Acc: 83.83% ± 4.56%
=== Watermarked Summary (grouped by g) ===
g=2  | Clean Acc: 52.37% | FGSM Acc: 43.10% | seeds: 100
g=3  | Clean Acc: 81.23% | FGSM Acc: 72.30% | seeds: 100
g=4  | Clean Acc: 87.73% | FGSM Acc: 78.93% | seeds: 100
g=5  | Clean Acc: 87.27% | FGSM Acc: 77.97% | seeds: 100
g=6  | Clean Acc: 92.70% | FGSM Acc: 84.67% | seeds: 100
g=7  | Clean Acc: 90.57% | FGSM Acc: 82.87% | seeds: 100
g=8  | Clean Acc: 91.87% | FGSM Acc: 83.17% | seeds: 100
g=9  | Clean Acc: 89.10% | FGSM Acc: 81.83% | seeds: 100
g=10 | Clean Acc: 93.33% | FGSM Acc: 86.27% | seeds: 100
